In [2]:
#import relavent libraries and packages
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import time

from tensorflow.keras.utils import to_categorical
from BinaryConnect import BinaryConnectCNN_SVHN
from BinaryConnect import custom_train_cnn
import time
import matplotlib.pyplot as plt
import scipy

In [4]:
import scipy.io
import numpy as np
import urllib.request
import io

def load_svhn_data_online(url):
    """ Load SVHN dataset from a URL """
    # Download and load the .mat file
    response = urllib.request.urlopen(url)
    content = response.read()
    data = scipy.io.loadmat(io.BytesIO(content))
    images = np.transpose(data['X'], [3, 0, 1, 2])
    labels = data['y'].flatten()
    labels[labels == 10] = 0  # Replace label 10 with 0
    return images, labels

# URLs to the SVHN dataset
train_url = 'http://ufldl.stanford.edu/housenumbers/train_32x32.mat'
test_url = 'http://ufldl.stanford.edu/housenumbers/test_32x32.mat'

# Load the training and test data
x_train, y_train = load_svhn_data_online(train_url)
x_test, y_test = load_svhn_data_online(test_url)


In [5]:
# Convert to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Compute the global mean and standard deviation
global_mean = np.mean(x_train, axis=(0, 1, 2))
global_std = np.std(x_train, axis=(0, 1, 2))

# Apply global contrast normalization
x_train = (x_train - global_mean) / (global_std + 1e-7)
x_test = (x_test - global_mean) / (global_std + 1e-7)

# Convert labels to one-hot encoding
num_classes = 10  # Adjust based on the dataset's specifics
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# ZCA Whitening
datagen = ImageDataGenerator(zca_whitening=True)
datagen.fit(x_train)


c:\Users\chris\anaconda3\envs\envTF210\lib\site-packages\keras\preprocessing\image.py:1446: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn(


In [6]:
#ensure using GPU to train the model to reduce training speed
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
#Trainingparameters
batch_size = 50
epochs = 500  # Reduced for faster training on subset
validation_split = 0.1
early_stopping_patience = 40


# Initialize lists to store metrics for plotting
stochastic_metrics = []
deterministic_metrics = []

# Train models with both deterministic and stochastic binarization
for deterministic in [False, True]:
    test_errors = []
    train_times = []
    trained_epochs = []

    # Create and compile the model
    model = BinaryConnectCNN_SVHN(deterministic)
    model.compile(
        optimizer='Adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Split data into training and validation sets
    val_size = int(len(x_train) * validation_split)
    x_val, y_val = x_train[-val_size:], y_train[-val_size:]
    x_train_subset, y_train_subset = x_train[:-val_size], y_train[:-val_size]

    # Start timing model training
    start_time = time.time()

    # Train the model using custom train function
    train_losses, val_accuracies = custom_train_cnn(model, x_train_subset, y_train_subset, x_val, y_val, epochs, batch_size, early_stopping_patience)

    # Store the training metrics for plotting
    history_dict = {
        'loss': train_losses,
        'val_accuracy': val_accuracies
    }

    if deterministic:
        deterministic_metrics.append(history_dict)
    else:
        stochastic_metrics.append(history_dict)

    # Record training time and epochs
    train_time = time.time() - start_time
    train_times.append(train_time)
    trained_epochs.append(epochs)  # Since custom_train doesn't return history

    # Evaluate model on test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    test_errors.append(1 - test_accuracy)

    # Store metrics

    print(
        f"Deterministic Binarization: {deterministic}\n"
        f"Average Test Error: {np.mean(test_errors)} +/- {np.std(test_errors)}\n"
        f"Average Training Time: {np.mean(train_times)} seconds\n"
        f"Average Number of Epochs: {np.mean(trained_epochs)}"
    )

    

Epoch 1/500
Epoch 1/500, Train Loss: 2.4937117099761963, Train Accuracy: 13.773282051086426, Val Loss: 2.479660987854004, Val Accuracy: 12.491467475891113
Epoch 2/500
Epoch 2/500, Train Loss: 2.3688371181488037, Train Accuracy: 14.97148609161377, Val Loss: 2.435250759124756, Val Accuracy: 14.716723442077637
Epoch 3/500
Epoch 3/500, Train Loss: 2.3361258506774902, Train Accuracy: 15.858763694763184, Val Loss: 2.477688789367676, Val Accuracy: 11.549488067626953
Epoch 4/500
Epoch 4/500, Train Loss: 2.3124709129333496, Train Accuracy: 16.365346908569336, Val Loss: 2.387118339538574, Val Accuracy: 16.368600845336914
Epoch 5/500
Epoch 5/500, Train Loss: 2.2951204776763916, Train Accuracy: 16.6868896484375, Val Loss: 2.3452792167663574, Val Accuracy: 15.631400108337402
Epoch 6/500
Epoch 6/500, Train Loss: 2.2822372913360596, Train Accuracy: 17.29205894470215, Val Loss: 2.32820200920105, Val Accuracy: 16.600683212280273
Epoch 7/500
Epoch 7/500, Train Loss: 2.268902063369751, Train Accuracy: 17

In [1]:
# Plotting
plt.figure(figsize=(10, 5))

# Plot for stochastic binarization
for history in stochastic_metrics:
    plt.plot(history['loss'], 'b--')  # Training loss
    plt.plot(history['val_accuracy'], 'b')  # Validation accuracy

# Plot for deterministic binarization
for history in deterministic_metrics:
    plt.plot(history['loss'], 'g--')  # Training loss
    plt.plot(history['val_accuracy'], 'g')  # Validation accuracy

plt.title('Training Curves on SVHN')
plt.xlabel('Number of Epochs Trained')
plt.ylabel('Validation Loss/ Accuracy')
plt.legend(['Stochastic Train Loss', 'Stochastic Val Accuracy',
            'Deterministic Train Loss', 'Deterministic Val Accuracy'], loc='upper right')
plt.show()

NameError: name 'plt' is not defined